In [1]:
from lightfm_recomender import LightfmRecomender
import os
%load_ext autoreload
%autoreload 2
new_directory = '/Users/vasevooo/projects/GameBuddy'
os.chdir(new_directory)
import pandas as pd 
import requests
import json
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
from lightfm import LightFM
from scipy.sparse import csr_matrix, save_npz, load_npz
import re
from bs4 import BeautifulSoup

/Users/vasevooo/projects/GameBuddy/steam/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/vasevooo/projects/GameBuddy/steam/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


### обновление предобработанных данных для модели 

In [7]:
games = pd.read_csv('data/game_info_with_scores.csv')
item_dict ={}
games = games[['appid', 'Name']].sort_values('appid').reset_index()
games['appid'] = games['appid'].astype(int)

for i in range(games.shape[0]):
    item_dict[int(games.loc[i,'appid'])] = games.loc[i,'Name']

In [8]:
## сохранение словаря по играм для дальнейшего использования

with open("data/for_models/item_dict.json", "w") as outfile:
    json.dump(item_dict, outfile, indent=4)

### Тест класса LigthFM Recomender

In [2]:
item_dict_path = 'data/for_models/item_dict.json'
gametime_df_path = 'data/final_gametime_df.csv'

lfr = LightfmRecomender(gametime_df_path, item_dict_path)

In [11]:
steam_url = 'https://steamcommunity.com/id/AVROORA'
steamid = 76561198156258305

In [13]:
new_user = lfr.get_user_games(steam_url)

In [14]:
matrix, df = lfr.get_csr_matrix_for_all_users(new_user)

In [15]:
## в приложении поставить 100 эпох. 1 для теста

model = lfr.fit_model(matrix, epochs=1)

In [7]:
lfr.sample_recommendation_user(model, df, steamid, threshold = 0, nrec_items = 10)

User: 76561198156258305
Recommended Items:
1- Portal 2
2- Terraria
3- Don't Starve Together
4- The Elder Scrolls V: Skyrim
5- Dying Light
6- Half-Life 2
7- The Witcher 2: Assassins of Kings Enhanced Edition
8- Garry's Mod
9- Tomb Raider
10- Darkest Dungeon®

 Known Likes:
1- Torchlight: Infinite
2- EA SPORTS™ FIFA 23
3- Vampire Survivors
4- V Rising
5- God of War
6- The Callisto Protocol™
7- Forza Horizon 4
8- ELDEN RING
9- Gunfire Reborn
10- Potion Craft: Alchemist Simulator
11- Gears Tactics
12- Sea of Thieves 2023 Edition
13- Apex Legends™
14- Total War: WARHAMMER III
15- Curse of the Dead Gods
16- Cyberpunk 2077
17- Destiny 2
18- Ori and the Will of the Wisps
19- Dota Underlords
20- Mortal Kombat 11
21- Among Us
22- Back 4 Blood
23- Age of Empires II: Definitive Edition
24- DOOM Eternal
25- Gloomhaven
26- Ring of Elysium
27- UnderMine
28- Slay the Spire
29- Disco Elysium - The Final Cut
30- Quake Champions
31- Realm Grinder
32- Graveyard Keeper
33- Hunt: Showdown
34- Dead Cells
35-

### cold start

In [16]:
df_cold_start = pd.DataFrame({'game_names': ['Counter-Strike', 'Team Fortress Classic', 
                                             'Day of Defeat', 
                                             'Half-Life: Source', 'Half-Life: Opposing Force'],
                   'rankings': [5, 5, 8, 7, 3]})

In [17]:
cold_start_user = lfr.get_user_games_cold_start(df_cold_start)

In [19]:
matrix_cs, df_cs = lfr.get_csr_matrix_for_all_users(cold_start_user)

In [20]:
## в приложении поставить 100 эпох. 1 для теста
model = lfr.fit_model(matrix_cs, epochs=1)

In [21]:
steamid_cold = 1234567891010102930 ## хардкод в скрипте
lfr.sample_recommendation_user(model, df_cs, steamid_cold, threshold = 0, nrec_items = 10)

User: 1234567891010102930
Recommended Items:
1- Counter-Strike: Global Offensive
2- Left 4 Dead 2
3- PAYDAY 2
4- Portal 2
5- Terraria
6- The Witcher® 3: Wild Hunt
7- Garry's Mod
8- Half-Life 2
9- Don't Starve Together
10- Tomb Raider

 Known Likes:
1- Half-Life: Source
2- Day of Defeat
3- Team Fortress Classic
4- Counter-Strike
